# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
### Replace each question mark with the appropriate value.
import numpy as np
# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = (X_train.shape[1],X_train.shape[2],X_train.shape[3])

# TODO: How many unique classes/labels there are in the dataset.
n_classes = np.unique(y_train).size

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
import random
# Visualizations will be shown in the notebook.
%matplotlib inline

def visualize(X_train, y_train) :
        indx = np.arange(y_train.shape[0])
        np.random.shuffle(indx)
        # create subplot and fill with images
        f, e = plt.subplots(10, 3)
        for i in range(10):
            for j in range(3):
                e[i, j].imshow(X_train[indx[j+10*i]])
                plt.setp(e[i,j].get_xticklabels(), visible=False)
                plt.setp(e[i,j].get_yticklabels(), visible=False)
        plt.show()
        
#visualize(X_train, y_train)


In [ ]:
   
def visualize_distribution(y_test, y_train):
    plt.figure(1)
    n, bins, patches = plt.hist(y_train, 70, facecolor='green', alpha=0.75)

    plt.xlabel('Train Label Classes')
    plt.ylabel('Train Label Samples Number')
    plt.title('Training Samples Distribution')
    plt.grid(True)

    plt.figure(2)
    n, bins, patches = plt.hist(y_test,70, facecolor='green', alpha=0.75)

    plt.xlabel('Test Label Classes')
    plt.ylabel('Test Label Samples Number')
    plt.title('Test Samples Distribution')
    plt.grid(True)
    plt.show()
    
#visualize_distribution(y_test, y_train)


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
from skimage import exposure

import time
from tqdm import tqdm

class DataPreprocessor(object) :
    
    def simple_rescale(self,image):
        image = image / 255. * 0.8 + 0.1
        return image

    def mean_substract(self,image):
        #np.mean(images, axis = 0,dtype=np.uint8)
        image -= np.mean(image, axis = 0,dtype=np.uint8)
        return image

    def feature_standardization(self, image):
        image = image / np.std(image, axis = 0)
        return image

    def exposure(self,image):
        return exposure.equalize_adapthist(image)

    def preprocess(self,image):
        #image = substract_noise(image)
        image = self.simple_rescale(image)
        image = self.mean_substract(image)
        image = self.exposure(image)
        return image


    def preprocess_batch(self,images):
        imgs = np.copy(images)
        for i in tqdm(range(images.shape[0])):
            imgs[i] = self.preprocess(images[i])
        return imgs
    '''
    def preprocess_batch(self,X):
        #Convert to grayscale, e.g. single Y channel
        X = 0.299 * X[:, :, :, 0] + 0.587 * X[:, :, :, 1] + 0.114 * X[:, :, :, 2]
        #Scale features to be in [0, 1]
        X = (X / 255.).astype(np.float32)

        # Apply localized histogram localization  
        for i in tqdm(range(X.shape[0])):
            X[i] = exposure.equalize_adapthist(X[i])
        
        # Add a single grayscale channel
        X = X.reshape(X.shape + (1,)) 
        return X
    '''

In [ ]:
import matplotlib.pyplot as plt
# Test preprocessing algorithm on one image
'''

dp = DataPreprocessor()     
plt.figure(1)
plt.imshow(X_train[10])
plt.figure(2)
X_train = dp.preprocess_batch(X_train)
plt.imshow(X_train[10])
plt.show()


X_train = dp.preprocess_batch(X_train)
X_test = dp.preprocess_batch(X_test)
'''


### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

# Data Preprocessing

The preprocess stage aims to :
  1.Normalize data

1.Data Normalization : in order to make the learning process performing better. The CNN learns its weights by adding gradient error vectors multiplied by the learning rate computed from backprop to weight matrices as the training process is running. so we end up with the following equation :
W = W + (learningrate * gradient descent)

Initial conditions : At the begining of the learning process, the input features values distributions are different for each one (feature). Thus the learning rate would correct in each dimension from one another. it might be over compensating a correction in one weight dimension while undercompensating in another.

The gradient descent find itself oscillating and unable to center or moving slowly  onto a better minima. So therefore, to reach a better minima and avoiding the oscillation (see above)  we need to normalize the input features values.

The scenario for preprocessing dataset is based on dataset normalization using three following methods to compare the average error (%) for each (http://ijssst.info/Vol-15/No-2/data/3251a024.pdf §IV. RESULTS AND DISCUSSIONS.)

The methods are :
    1.Simple Rescale
    2.Substract Mean 
    


Each normalization methods is implemented by a "normalized" pipeline (see "class DataPreProcessor" bellow).
The normalization is part of the preprocess stage and should be applied after the Data augmentation stage (see later "class DataAugmenter") if exists.




In [ ]:
#apply_preprocessing(X_train, y_train,True) 

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

# Data Augmentation and Spliting

Data augmentation is done to artificially increase the size of the dataset. Various affine transformations are applied to slightly perturb each image so that the network will receive a different variation of each image on every iteration.

Random rotations between -10 and 10 degrees.
Random translation between -10 and 10 pixels in any direction.
Random zoom between factors of 1 and 1.3.
Random shearing between -25 and 25 degrees.
Bool choice to invert colors.
Sobel edge detector applied to 1/4 of images.


In [ ]:
from skimage import transform, filters, exposure, img_as_float
from skimage.transform import rescale
from random import randint,uniform

class DataAugmenter(object):
    def fast_warp(self, img, tf, output_shape, mode='wrap'):
        return transform._warps_cy._warp_fast(img, tf.params,
                                          output_shape=output_shape, mode=mode)

    def augment(self,images):
        # set empty copy to hold augmented images so that we don't overwrite
        #print(images.shape)
        #print(images[1][:,:][:,:])
        images_aug = np.empty(shape = (images.shape[0], images.shape[1],images.shape[2],images.shape[3]),
                               dtype = 'float32')
        print(images_aug[0].shape)

        # random rotations betweein -10 and 10 degrees
        dorotate = randint(-10,10)

        # random translations
        trans_1 = randint(-10,10)
        trans_2 = randint(-10,10)

        # random zooms
        zoom = uniform(1, 1.3)

        # shearing
        shear_deg = uniform(-25, 25)

        # set the transform parameters for skimage.transform.warp
        # have to shift to center and then shift back after transformation otherwise
        # rotations will make image go out of frame
        center_shift   = np.array((32, 32)) / 2. - 0.5
        tform_center   = transform.SimilarityTransform(translation=-center_shift)
        tform_uncenter = transform.SimilarityTransform(translation=center_shift)

        tform_aug = transform.AffineTransform(rotation = np.deg2rad(dorotate),
                                              scale =(1/zoom, 1/zoom),
                                              shear = np.deg2rad(shear_deg),
                                              translation = (trans_1, trans_2))

        tform = tform_center + tform_aug + tform_uncenter

        #output = numpy.empty(images.shape[1],images.shape[2],images.shape[3]),dtype=numpy.float32)
        
        for j in range(images.shape[0]):
            for channel in range(images.shape[3]):
                images_aug[j][:, :,channel] = self.fast_warp(
                                                    images[j][:, :,channel],
                                                    tform,
                                                    output_shape=(images.shape[1],images.shape[2]))
        
        
        '''
        # use sobel edge detector filter on one quarter of the images
        indices_sobel = np.random.choice(images_aug.shape[0],
                                         images_aug.shape[0] / 4, replace = False)
        for k in indices_sobel:
            for channel in range(images.shape[3]):
                print(rescale(images_aug[k][:, :,channel],.5))
                images_aug[k][:, :,channel] = filters.sobel(rescale(images_aug[k][:, :,channel],.5))
        '''
        # invert half of the images
        indices_invert = np.random.choice(images_aug.shape[0],
                                          images_aug.shape[0] / 2, replace = False)
        for l in indices_invert:
            for channel in range(images.shape[3]):
                img = images_aug[l][:, :,channel]
                images_aug[l][:, :,channel] = np.absolute(img - np.amax(img))
            
        return images_aug

In [ ]:
#from itertools import groupby
#label_per_class = np.array([(k,len(list(v))) for k,v in groupby(y_train)])
#print(label_per_class)

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

def augment(X_train, y_train):
    da = DataAugmenter()
    X_valid = da.augment(X_train)
    y_valid = y_train
    plt.figure(1)
    plt.imshow(X_train[2045])
    plt.figure(2)
    #plt.imshow(X_valid[2045])
    plt.show()
    return X_valid,y_valid
X_validation, y_validation = augment(X_train, y_train)

In [ ]:
from sklearn.cross_validation import train_test_split
# We will use 80% for training and 20% for validation
def split_data(x, y):
    data_train, data_valid, labels_train, labels_valid = train_test_split(x, y, test_size=0.20, random_state=42)
    print("Shape of training examples = ", data_train.shape)
    print("Shape of validation examples = ", data_valid.shape)
    return data_train, data_valid, labels_train, labels_valid

#X_train, X_validation, y_train, y_validation = split_data(X_train, y_train)

In [ ]:
import tensorflow as tf
def accuracy(predictions, labels):
      return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

X_validation, y_validation = augment(X_train, y_train)

train_one_hot_labels = (np.arange(n_classes) == y_train[:,None]).astype(np.float32)
valid_one_hot_labels = (np.arange(n_classes) == y_validation[:,None]).astype(np.float32)
test_one_hot_labels = (np.arange(n_classes) == y_test[:,None]).astype(np.float32)


print('Training set', X_train.shape, train_one_hot_labels.shape)
print('Validation set', X_validation.shape, valid_one_hot_labels.shape)
print('Test set', X_test.shape, test_one_hot_labels.shape)


### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

My model uses LeNet Lab as starting point :
1/Convolutional Layer Input = 32x32x3 (3 channels), stride=1, filter=(5,5,3,10), Output = 28x28x10
2/Activation Layer 
3/Pooling Layer Input = 28x28x10, stride=1, kernel=(1,2,2,1), Output = 14x14x10
4/Convolutional Layer Input = 14x14x10, stride=(1, 1, 1, 1), filter=(5, 5, 10, 20) Output = 10x10x20
5/Activation Layer 
6/Pooling Layer Input = 10x10x20,stride=(1, 2, 2, 1) kernel=(1, 2, 2, 1), Output = 5x5x20
7/Flatten input = 5x5x20, output = 500
8/Fully Connected with added dropout input = 500, output = 160
9/Activation 
10/Fully Connected with added dropout Layer input = 120, output = 60
11/Activation 
12/fully connected with added dropout layer input = 84, outpput= 43

In [ ]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 0: Convolutional. Input = 32x32x1. Output = 32x32x3.
    conv0_W = tf.Variable(tf.truncated_normal(shape=(1, 1, 3, 3), mean = mu, stddev = sigma))
    conv0_b = tf.Variable(tf.zeros(3))
    conv0   = tf.nn.conv2d(x, conv0_W, strides=[1, 1, 1, 1], padding='VALID') + conv0_b

    # SOLUTION: Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x32.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 32), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(32))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x32. Output = 14x14x32.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    conv1 = tf.nn.dropout(conv1, 0.9)
  
    # SOLUTION: Layer 2: Convolutional. Input = 14x14x32. Output = 10x10x64.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 32, 64), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(64))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.dropout(conv2, 0.8)
    
    '''
    # SOLUTION: Pooling. Input = 10x10x64. Output = 5x5x64
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    conv2 = tf.nn.dropout(conv2, 0.5)
    '''
    
    # SOLUTION: Layer 3: Convolutional. Input = 10x10x64. Output = 5x5x128
    conv3_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 64, 128), mean = mu, stddev = sigma))
    conv3_b = tf.Variable(tf.zeros(128))
    conv3  = tf.nn.conv2d(conv2, conv3_W, strides=[1, 1, 1, 1], padding='VALID') + conv3_b
    
    # SOLUTION: Activation.
    conv3 = tf.nn.relu(conv3)
    conv3 = tf.nn.dropout(conv3, 0.7)
    
    '''
    # SOLUTION: Pooling. Input = 10x10x64. Output = 5x5x64
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    conv3 = tf.nn.dropout(conv3, 0.5)
    '''
    
    # SOLUTION: Flatten. Input = 14x14x32. Output = 4608.
    fc0   = flatten(conv3)
    
    # SOLUTION: Layer 4: Fully Connected. Input = 1024. Output = 1024.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(4608, 1024), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(1024))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, 0.5)
    '''
    # SOLUTION: Layer 4: Fully Connected. Input = 1024. Output = 1024.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(1024, 1024), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(1024))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, 0.5)
    '''
    
    # SOLUTION: Layer 5: Fully Connected. Input = 1024. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(1024, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc1, fc3_W) + fc3_b
    #logits = tf.nn.dropout(logits, 0.5)
    
    return logits

In [ ]:
### Train your model here.io
### Feel free to use as many code cells as needed.
rate = 0.0001
EPOCHS = 50
BATCH_SIZE = 100

x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)
logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)

# Optimizer.
optimizer = tf.train.AdamOptimizer(learning_rate = rate)

training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return (total_accuracy / num_examples)

In [86]:
from sklearn.utils import shuffle
import tensorflow as tf
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    print("Preprocessing Train Data...")
    print()
    dp = DataPreprocessor()     
    X_train = dp.preprocess_batch(X_train)
    
    print("Training Model...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            accuracy = sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy *100))
        print()
        
    saver.save(sess, './lenano')
    print("Model saved")

  0%|          | 0/39209 [00:00<?, ?it/s]/home/naaman/anaconda3/envs/CTSC/lib/python3.5/site-packages/scikit_image-0.12.3-py3.5-linux-x86_64.egg/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint16
  "%s to %s" % (dtypeobj_in, dtypeobj))
  0%|          | 13/39209 [00:00<05:10, 126.24it/s]

Preprocessing Train Data...



100%|██████████| 39209/39209 [04:56<00:00, 132.05it/s]


Training Model...

EPOCH 1 ...
Validation Accuracy = 5.264

EPOCH 2 ...
Validation Accuracy = 4.721

EPOCH 3 ...
Validation Accuracy = 4.685

EPOCH 4 ...
Validation Accuracy = 4.749

EPOCH 5 ...
Validation Accuracy = 4.588

EPOCH 6 ...
Validation Accuracy = 4.277

EPOCH 7 ...
Validation Accuracy = 4.915

EPOCH 8 ...
Validation Accuracy = 4.695

EPOCH 9 ...
Validation Accuracy = 4.649

EPOCH 10 ...
Validation Accuracy = 4.884

EPOCH 11 ...
Validation Accuracy = 4.428

EPOCH 12 ...
Validation Accuracy = 4.420

EPOCH 13 ...
Validation Accuracy = 4.647

EPOCH 14 ...
Validation Accuracy = 5.022

EPOCH 15 ...
Validation Accuracy = 4.644

EPOCH 16 ...
Validation Accuracy = 4.680

EPOCH 17 ...
Validation Accuracy = 4.264

EPOCH 18 ...
Validation Accuracy = 4.904

EPOCH 19 ...
Validation Accuracy = 4.593

EPOCH 20 ...
Validation Accuracy = 4.565

EPOCH 21 ...
Validation Accuracy = 4.563

EPOCH 22 ...
Validation Accuracy = 4.333

EPOCH 23 ...
Validation Accuracy = 3.979

EPOCH 24 ...
Validation 

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**
Train library is wrapper around tensorflow methods to encapsulate all of the training procedures of the model in one single method called train_model. This method has the following attributes:

1/ rate = 0.001
2/ EPOCHS = 10
3/ BATCH_SIZE = 128
4/ Hyperparameters : 
    4.1/ Conv layer #1 
            Number of filters = 128
            filter size = (5,5)
    4.2/ Relu Layer #1
    4.3/ Flatten
            
    4.4/ FC 1024
    4.5/ FC 1024
    4.6/ FC 43
            


implement the following workflow :
*/ Training data is drouped out using a parameter keep_prob = 0.5 for the validation training step and keep_prob=1.0 for the testing step
*/ Training data is normalized and augmented
*/for each batch of 128 examples, 
    */trained using an Adam optimizer because it uses Kingma and Ba's Adam algorithm to control the learning rate. Adam offers several advantages over the simple tf.train.GradientDescentOptimizer. http://stats.stackexchange.com/questions/184448/difference-between-gradientdescentoptimizer-and-adamoptimizer-tensorflow
    
    */Creates a validation dataset using 30% of the size of training dataset and the data augementer library. The advantages of this technique is taking a copy of each choosen training example and augment it which leads to not reduce the training dataset size. To do that for each epochs, the starting element of the validation dataset is calculated randomly.
    */Validation accuracy is then calculated and displayed.
    
   

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

The approach taken to solve this problem is a derived architecture of LeNet. 
I tried first the following architecture : Conv/Relu/Pool/Relu/Conv/Relu/Pool/Relu/Flatten/FC/FC/FC. The results were poor after many attemps of tweaking the Hyperparamters accuracy was under 50% after many trials 
So i changed the network architecture by samplifying it. I kept only one conv layer and three FCs layers. I modify three times  the batch size until 150 which make the validation accuracy over 89% local minima with no stucking to. The final parameters are :
learning rate = 0.001
Batch size = 150
Epochs = 100



---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

def load_images(path):

    imgnames = ['01.jpg', '02.jpg', '03.jpg', '04.jpg', '05.jpg']

    images = np.float32(np.zeros((10,32,32,3)))
    
    for idx,name in enumerate(imgnames):
        image = mpimg.imread(path + name)
        #image = image.resize((int(width),int(height)),Image.ANTIALIAS)
        print(image.shape)
        images[idx-1]=image
        plt.imshow(image)
        plt.show()
    
    #images =  np.array(images)
    return images   
    
#images = load_images('images/')

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**


In [ ]:
## Run the predictions here.
### Feel free to use as many code cells as needed.
images = load_images('images/')
print('images : ',images.shape)
graph = tf.Graph()
     
        
with tf.Session('', tf.Graph()) as sess:
            sess.run(tf.global_variables_initializer())
            saver = tf.train.import_meta_graph('./lenet.meta')
            #saver.restore(sess, tf.train.latest_checkpoint('.'))
            ckpt = str(tf.train.latest_checkpoint('.')) #tf.train.get_checkpoint_state('.')
            print(ckpt)
            if ckpt:
                saver.restore(sess, ckpt)
                print('Restored!')
                x = tf.placeholder(tf.float32, (10,32, 32, 3))
                keep_probe = tf.placeholder(tf.float32)
                model = Model()
                model.build(x,43,keep_probe)
                prediction = tf.nn.softmax(model.logits)

                with tf.Session() as session:
                    new_predictions = session.run(prediction, feed_dict={x: images})
                    top_k = session.run(tf.nn.top_k(new_predictions, 5, sorted=True))
                    print("Predicted Labels")
                    print(np.argmax(new_predictions, 1))

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.